# Lab-10-4-2 ImageFolder2

## Import

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from matplotlib.pyplot import imshow

## cuda

In [2]:
torch.cuda.is_available()

False

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

## Data Loader

In [4]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(
    root='./images/cat_dog/train_resized', transform=trans)

In [5]:
for num, value in enumerate(train_data):
    data, label = value
    print(data.shape)
    break

torch.Size([3, 64, 64])


In [6]:
data_loader = DataLoader(dataset=train_data,
                         batch_size=8,
                         shuffle=True,
                         num_workers=2
                        )

## Model

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5), 
            nn.ReLU(), 
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 5), 
            nn.ReLU(), 
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*13, 120), 
            nn.ReLU(), 
            nn.Linear(120, 2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.layer3(out)
        return out

#### Output size check

In [8]:
net = CNN().to(device)
random_input = torch.Tensor(3, 3, 64, 64)
test_out = net(random_input)

```
torch.Size([3, 3, 64, 64])
torch.Size([3, 6, 30, 30])
torch.Size([3, 16, 13, 13])
torch.Size([3, 2704])
torch.Size([3, 2])
```

In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)

In [10]:
total_batch = len(data_loader)
print('>>> Learning Start <<<')

epochs = 5
for epoch in range(epochs):
    avg_loss = 0.0
    
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        out = net(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_loss += loss / total_batch
    
    print("[Epoch {}] Loss: {}".format(epoch+1, avg_loss))

print('>>> Learning Finished <<<')

>>> Learning Start <<<
[Epoch 1] Loss: 0.6993120312690735
[Epoch 2] Loss: 0.6934748888015747
[Epoch 3] Loss: 0.6941900253295898
[Epoch 4] Loss: 0.6945840716362
[Epoch 5] Loss: 0.6941583752632141
>>> Learning Finished <<<


## Save and Load Models

In [11]:
torch.save(net.state_dict(), "./models/cat_dog.pth")

In [12]:
new_net = CNN().to(device)

In [13]:
new_net.load_state_dict(torch.load('./models/cat_dog.pth'))

<All keys matched successfully>

In [14]:
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.1620, -0.0641, -0.0839, -0.0908,  0.0239], grad_fn=<SelectBackward>)
tensor([-0.1620, -0.0641, -0.0839, -0.0908,  0.0239], grad_fn=<SelectBackward>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]])

## Test

In [15]:
trans=torchvision.transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

test_data = torchvision.datasets.ImageFolder(
    root='./images/cat_dog/test', transform=trans)

In [16]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [17]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 0.5199999809265137
